In [3]:
import requests

In [108]:
list_of_schools = ['Severn Primary','St. Mary\'s R.C. Primary School']

In [4]:
toppage = requests.get("http://mylocalschool.wales.gov.uk/Schools/SchoolSearch?lang=en")

In [5]:
from bs4 import BeautifulSoup

In [44]:
soup = BeautifulSoup(toppage.content, 'html.parser')

In [72]:
school_links = soup.select("a[href*=/School]")[2:]

In [102]:
schools = [list(school.children)[0] for school in school_links]

In [103]:
school_URLs = ["http://mylocalschool.wales.gov.uk" + school['href'] for school in school_links]

In [105]:
schoolURL_dict = {}

In [185]:
for s,school in enumerate(schools):
    schoolURL_dict[school] = school_URLs[s]

In [186]:
school_soup = {}
for school in list_of_schools:
    school_pages = requests.get(schoolURL_dict[school])
    school_soup[school] = BeautifulSoup(school_pages.content, 'html.parser')

In [309]:
def make_stat_dict(schoolURL):
    school_pages = requests.get(schoolURL)
    school_soup = BeautifulSoup(school_pages.content, 'html.parser')
    summary = school_soup.find_all('div', class_="summaryBox")
    stat_dict = {}
    for i in range(8):
        stat = ''.join(char for char in list(summary[i].children)[1].getText() if char.isalnum() or char=='.')
        if stat.isnumeric(): stat=float(stat)
        stat_name = (''.join(char for char in list(summary[i].children)[3].getText() if char.isalnum() or char==' ')).strip()
        if '%' in list(summary[i].children)[1].getText(): stat_name += "(%)"
        stat_dict[stat_name] = stat
    return stat_dict

In [310]:
the_grand_dict = {}
for school in list_of_schools:
    the_grand_dict[school] = make_stat_dict(schoolURL_dict[school])

In [311]:
the_grand_dict['Severn Primary']

{'Attendance during the year Primary only(%)': '93.9',
 'Free school meals FSM  3 year average Primary only(%)': 24.0,
 'Number of Pupils 2017': 482.0,
 'Pupil Teacher Ratio PTR Primary only': '20.9',
 'Pupils achieving the expected outcome in the Foundation Phase areas of learning(%)': '85.5',
 'Pupils who have reached the expected level  Core subject indicator Key Stage 2(%)': '87.5',
 'School budget per pupil': 3947.0,
 'Support Category': 'Green'}